In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [2]:
# album and artist
icm = dr.getICM_csr(albums=True,artists=True,tags=False)
icm = normalize(icm,axis=0)
model = BM25Recommender()
model.fit(icm)
s = model.similarity
r_al_ar = urm*s.T #or s
r_al_ar = normalize(r_al_ar, axis=0)
if evaluation: score = dr.evaluateMAP(r_al_ar)

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0786


In [3]:
# tags
icm = dr.getICM_csr(albums=False,artists=False,tags=True)
icm = normalize(icm,axis=1)   # little improvment? or just overfitting?
model = BM25Recommender()
model.fit(icm)
s = model.similarity
r_tag = urm*s #######wuuuut no urm
r_tag = normalize(r_tag, axis=0)
if evaluation: score = dr.evaluateMAP(r_tag)

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0370


In [4]:
#cf user based
urm2 = urm.copy()
model = BM25Recommender()
model.fit(urm2)
s = model.similarity
r_cfub = s * urm2
r_cfub = normalize(r_cfub, axis=1) 
if evaluation: score = dr.evaluateMAP(r_cfub)

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0627


In [5]:
#collaborative filtering item based
m_cf_ib = CF_IB_asymmetric_cosine(urm, k=200, alpha=0.30, q=1, m=0)
s = m_cf_ib.s
r_cfib = urm*s

if evaluation: score = dr.evaluateMAP(r_cfib)

Start asymmetric cosine item-based model..
Keep only k-similar item..
Building sparse matrix..
Start building rating matrix..
Normalizing rating matrix..
Model asymmetric cosine item-based done
User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0721


In [7]:
# reduce matrix
r_cfib_asy = normalize(r_cfib,axis=1) #4.75
r_cfib_less = normalize(r_cfib,axis=0)#2.75 #######axis 0 better in sum

r_al_ar = dr.reduceRM(r_al_ar) 
r_tag = dr.reduceRM(r_tag)
r_cfib = dr.reduceRM(r_cfib) 
r_cfub = dr.reduceRM(r_cfub)


In [9]:
r_tot = r_al_ar + 2.75 * r_cfib_less + 0.4 * r_tag
if evaluation: score = dr.evaluateMAP(r_tot) #2.75

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0989


In [15]:
r_tot2 = r_tot + 1.3 * r_cfub
if evaluation: score = dr.evaluateMAP(r_tot2) 

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0992


In [19]:
r_bpr = dr.reduceRM(urm * dr.load_sparse_csr("./DumpData/sim_bpr_10e_100k2.npz"))

if evaluation: score = dr.evaluateMAP(r_bpr)

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0637


In [20]:
r_tot3 = r_tot2 + 14*r_bpr

if evaluation: score = dr.evaluateMAP(r_tot3)

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0990


In [21]:
r_tot4 =r_tot3 + 0.35*r_cfib_asy

if evaluation: score = dr.evaluateMAP(r_tot4)

User 0 of 10000
User 2500 of 10000
User 5000 of 10000
User 7500 of 10000
Recommender performance is: MAP = 0.0985


In [14]:

dr.buildSubmissionFile(recommendation=rec, file_name="ci_si_prova")

Creating submission file in: ./Submissions/ci_si_prova_with_trick
Done


In [355]:
urm2 = urm.copy()
model = BM25Recommender()
model.fit(urm2.T)
s = model.similarity
r = urm2*s.T
r = normalize(r, axis=1)
if evaluation: score = dr.evaluateMAP(r)
print (score)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0673
0.0673044166667


In [357]:
r_totcf = r_cfub + r
r_totcf = normalize(r_totcf, axis=1) 
dr.evaluateMAP(r_totcf)

User 0 of 10000
User 2000 of 10000
User 4000 of 10000
User 6000 of 10000
User 8000 of 10000
Recommender performance is: MAP = 0.0737


0.073743805555555889

In [148]:
def norm1(m):
    return normalize(m,axis=0)
def norm2(m):
    return normalize(m,axis=1)